In [1]:
!pip install --upgrade "protobuf>=3.20.0,<4.0.0" "sentence-transformers>=3.0.0" transformers scikit-learn faiss-gpu-cu11 onnxruntime-gpu optimum[onnxruntime-gpu] accelerate datasets psutil requests tqdm flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 82.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 29.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 k

In [2]:
import torch

print("CUDA Configuration:")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
print(f"✓ CUDA device count: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"  - GPU {i}: {torch.cuda.get_device_name(i)} - {torch.cuda.get_device_properties(i).total_memory // 1024**3}GB")

# Verify FAISS GPU
try:
    import faiss
    print(f"\n✓ FAISS version: {faiss.__version__}")
    print(f"✓ FAISS GPU support: Available")
except ImportError:
    print("✗ FAISS not available")

CUDA Configuration:
✓ CUDA available: True
✓ CUDA device count: 2
  - GPU 0: Tesla T4 - 14GB
  - GPU 1: Tesla T4 - 14GB

✓ FAISS version: 1.12.0
✓ FAISS GPU support: Available


In [3]:
!hf download Qdrant/all_miniLM_L6_v2_with_attentions; hf download Qdrant/all-MiniLM-L6-v2-onnx; hf download Qdrant/bm25; hf download BAAI/bge-m3; hf download jinaai/jina-code-embeddings-1.5b; hf download jinaai/jina-embeddings-v4; hf download jinaai/jina-reranker-v3; hf download Qwen/Qwen3-Embedding-0.6B

Fetching 9 files:   0%|                                   | 0/9 [00:00<?, ?it/s]Downloading '.gitattributes' to '/root/.cache/huggingface/hub/models--Qdrant--all_miniLM_L6_v2_with_attentions/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'

.gitattributes: 1.52kB [00:00, 5.97MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--Qdrant--all_miniLM_L6_v2_with_attentions/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b
Fetching 9 files:  11%|███                        | 1/9 [00:00<00:02,  3.26it/s]Downloading 'special_tokens_map.json' to '/root/.cache/huggingface/hub/models--Qdrant--all_miniLM_L6_v2_with_attentions/blobs/9bbecc17cabbcbd3112c14d6982b51403b264bfa.incomplete'

special_tokens_map.json: 100%|█████████████████| 695/695 [00:00<00:00, 5.80MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--Qdrant--all_miniLM_L6_v2_with_attentions/blobs/9bbecc17cabbcbd3112c14d6982b51403b264bfa

tokenizer.json: 0.00B [00:00, ?B/s]

config.json

In [ ]:
!hf download Qdrant/all_miniLM_L6_v2_with_attentions; hf download Qdrant/all-MiniLM-L6-v2-onnx; hf download Qdrant/bm25; hf download BAAI/bge-m3; hf download jinaai/jina-code-embeddings-1.5b; hf download jinaai/jina-embeddings-v4; hf download jinaai/jina-reranker-v3; hf download Qwen/Qwen3-Embedding-0.6B

In [4]:
!git clone https://github.com/eldon31/rag_clean.git
%cd /kaggle/working/rag_clean

Cloning into 'rag_clean'...
remote: Enumerating objects: 8798, done.
remote: Counting objects: 100% (4025/4025), done.
remote: Compressing objects: 100% (1052/1052), done.
remote: Total 8798 (delta 3117), reused 3808 (delta 2907), pack-reused 4773 (from 1)
Receiving objects: 100% (8798/8798), 87.08 MiB | 29.28 MiB/s, done.
Resolving deltas: 100% (5977/5977), done.
/kaggle/working/rag_clean


In [5]:
!python scripts/embed_collections_v5.py

2025-10-21 00:01:01.723986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761004861.957622     362 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761004862.020301     362 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Multiple distributions found for package optimum. Picked distribution: optimum-onnx
✓ Successfully imported UltimateKaggleEmbedderV4
ULTIMATE KAGGLE EMBEDDER V5 - BATCH RUNNER
Optimized for V5 Unified Chunker Output

✓ Logging initialized
✓ Python executable: /usr/bin/python3
✓ Kaggle environment: True
✓ CUDA available: True
✓ CUDA device count: 2
  - GPU 0: Tesla T4
  - GPU 1: Tesla T4

Parsing command-line arguments...
✓ Arguments 